# Исследование опроса клиентов телекомунникационной компании

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

**Задачи исследования** - ответить на вопросы:
1. Как распределены участники опроса по возрасту, полу и возрасту? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
2. Какие группы пользователей наиболее лояльны к сервису? Какие менее?
3. Какой общий NPS среди всех опрошенных?
4. Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

**Ход исследования**:
1. Подключение к базе
2. Выгрузка данных 
   2.1. Создание SQL запроса
   2.2. Вывод данных
   2.3. Изучение данных
   2.4. Сохранение данных в csv файл
3. Создание дашборда в Tableau
4. Ответы на вопросы с выводами  в презентации


## Подключение к базе

In [1]:
# импорт нужных библиотек
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db' # путь к файлу
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

## Выгрузка данных 

### Создание SQL запроса

In [3]:
query = """

SELECT user.user_id,
       user.lt_day,
       CASE
           WHEN user.lt_day <= 365 THEN 'новый'
           WHEN user.lt_day > 365 THEN 'не новый'
       END AS is_new,
       user.age,
       CASE 
           WHEN user.gender_segment = 1 THEN 'женщина'
           WHEN user.gender_segment = 0 THEN 'мужчина'
       END AS gender_segment,
       user.os_name,
       user.cpe_type_name,
       location.country,
       location.city,
       substr(age_segment.title, 4) AS age_segment,
       substr(traffic_segment.title, 4) AS traffic_segment,
       substr(lifetime_segment.title, 4) AS lifetime_segment,
       user.nps_score,
       CASE 
           WHEN user.nps_score < 7 THEN 'критики'
           WHEN user.nps_score > 6 AND user.nps_score < 9 THEN 'нейтралы'
           WHEN user.nps_score > 8 THEN 'сторонники'
       END AS nps_group
FROM user AS user
JOIN location AS location ON user.location_id = location.location_id
JOIN age_segment AS age_segment ON user.age_gr_id = age_segment.age_gr_id
JOIN traffic_segment AS traffic_segment ON user.tr_gr_id = traffic_segment.tr_gr_id
JOIN lifetime_segment AS lifetime_segment ON user.lt_gr_id = lifetime_segment.lt_gr_id;

"""

### Вывод данных

In [4]:
df = pd.read_sql(query, engine)
df.head(15)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,не новый,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,не новый,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,не новый,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники
3,A004TB,4190,не новый,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,сторонники
4,A004XT,1163,не новый,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,сторонники
5,A005O0,5501,не новый,42.0,женщина,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,критики
6,A0061R,1236,не новый,45.0,мужчина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,сторонники
7,A009KS,313,новый,35.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,сторонники
8,A00AES,3238,не новый,36.0,женщина,ANDROID,SMARTPHONE,Россия,СанктПетербург,35-44,1-5,36+,10,сторонники
9,A00F70,4479,не новый,54.0,женщина,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,сторонники


### Изучение данных

Мы извлекли нужные нам данные и имеем следующую информацию:

* `user_id` -	идентификатор клиента
* `lt_day` - количество дней «жизни» клиента
* `is_new` - поле хранит информацию о том, является ли клиент новым
* `age` -	возраст
* `gender_segment` - пол 
* `os_name` -	тип операционной системы
* `cpe_type_name` - тип устройства
* `country` - страна проживания
* `city` - город проживания
* `age_segment` - возрастной сегмент
* `traffic_segment` - сегмент по объёму потребляемого трафика
* `lifetime_segment` - сегмент по количеству дней «жизни»
* `nps_score` - оценка клиента в NPS-опросе
* `nps_group` - поле хранит информацию о том, к какой группе относится оценка клиента в опросе

### Сохранение данных

In [5]:
# сохраним данные в csv формате
df.to_csv('telecomm_csi_tableau.csv', index=False)

## Создание дашборда в Tableau

*Ссылка на дашборд* : https://public.tableau.com/app/profile/.85962202/viz/NPS_V2/Dashboard1?publish=yes

## Ответы на вопросы с выводами в презентации

*Ссылка на презентацию* : https://drive.google.com/file/d/15HikldRn2kwTAP_9Hsd7Pvcwapb0Knw4/view?usp=sharing